In [1]:
# import load_dotenv
from dotenv import load_dotenv

# load env
load_dotenv()

True

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [5]:
from pydantic_ai import Agent, RunContext

roulette_agent = Agent(  
    'openai:gpt-4o',
    deps_type=int,
    result_type=bool,
    system_prompt=(
        'Use the `roulette_wheel` function to see if the '
        'customer has won based on the number they provide.'
    ),
)


@roulette_agent.tool
async def roulette_wheel(ctx: RunContext[int], square: int) -> str:  
    """check if the square is a winner"""
    return 'winner' if square == ctx.deps else 'loser'

In [6]:
# Run the agent
success_number = 18  
result = roulette_agent.run_sync('Put my money on square eighteen', deps=success_number)
print(type(result))
print(result)
print(result.data)
#> True

<class 'pydantic_ai.agent.AgentRunResult'>
AgentRunResult(data=True)
True
Getting weather info for 26.3683, -80.1289
Getting weather info for 25.7617, -80.1918
<class '__main__.User'>
User(name='Anne')
Getting weather info for 26.3683, -80.1289


In [7]:
result = roulette_agent.run_sync('I bet five is the winner', deps=success_number)
print(result.data)
#> False

False


In [8]:
from pydantic_ai import Agent

agent = Agent('openai:gpt-4o')

result = await agent.run('What is the capital of France?')
print(result.data)
#> Paris

async with agent.run_stream('What is the capital of the UK?') as response:
    print(await response.get_data())
    #> London

The capital of France is Paris.
The capital of the United Kingdom is London.


In [10]:
from pydantic_ai import Agent
from pydantic_ai.exceptions import UsageLimitExceeded
from pydantic_ai.usage import UsageLimits

agent = Agent('anthropic:claude-3-7-sonnet-latest')

result_sync = agent.run_sync(
    'What is the capital of Italy? Answer with just the city.',
    usage_limits=UsageLimits(response_tokens_limit=10),
)
print(result_sync.data)
print(result_sync.usage())

Rome
Usage(requests=1, request_tokens=20, response_tokens=4, total_tokens=24, details=None)


In [12]:
from pydantic_ai import Agent
from dotenv import load_dotenv

agent = Agent('anthropic:claude-3-7-sonnet-latest', system_prompt='Be as concise as possible, reply with one sentence.')

message_history = []
while True:
    current_message = input('You: ')
    if current_message == 'quit':
        break
    result = agent.run_sync(current_message, message_history=message_history)
    message_history = result.new_messages()
    print(type(message_history))
    print(message_history)
    print(result.data)

You:  When was the US declaration of Independence?


<class 'list'>
[ModelRequest(parts=[SystemPromptPart(content='Be as concise as possible, reply with one sentence.', timestamp=datetime.datetime(2025, 4, 10, 10, 38, 43, 180977, tzinfo=datetime.timezone.utc), dynamic_ref=None, part_kind='system-prompt'), UserPromptPart(content='When was the US declaration of Independence?', timestamp=datetime.datetime(2025, 4, 10, 10, 38, 43, 180981, tzinfo=datetime.timezone.utc), part_kind='user-prompt')], kind='request'), ModelResponse(parts=[TextPart(content='The US Declaration of Independence was signed on July 4, 1776.', part_kind='text')], model_name='claude-3-7-sonnet-20250219', timestamp=datetime.datetime(2025, 4, 10, 10, 38, 43, 821153, tzinfo=datetime.timezone.utc), kind='response')]
The US Declaration of Independence was signed on July 4, 1776.


You:  quit


In [14]:
import requests
from pydantic_ai import Agent

agent = Agent('anthropic:claude-3-7-sonnet-latest', system_prompt='Be as concise as possible, reply with one sentence.')

@agent.tool_plain
def get_weather_info(latitude: float, longitude: float) -> str:
    print(f"Getting weather info for {latitude}, {longitude}")
    url = f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,weather_code"
    response = requests.get(url)
    return response.json()

message_history = []
while True:
    current_message = input('You: ')
    if current_message == 'quit':
        break
    result = agent.run_sync(current_message, message_history=message_history)
    message_history = result.new_messages()
    print(type(message_history))
    print(message_history)
    print(type(result))
    print(result.data)

You:  What's the temperature in Boca Raton, US today


<class 'list'>
[ModelRequest(parts=[SystemPromptPart(content='Be as concise as possible, reply with one sentence.', timestamp=datetime.datetime(2025, 4, 10, 10, 50, 4, 377082, tzinfo=datetime.timezone.utc), dynamic_ref=None, part_kind='system-prompt'), UserPromptPart(content="What's the temperature in Boca Raton, US today", timestamp=datetime.datetime(2025, 4, 10, 10, 50, 4, 377100, tzinfo=datetime.timezone.utc), part_kind='user-prompt')], kind='request'), ModelResponse(parts=[TextPart(content='Let me check the current temperature in Boca Raton for you.', part_kind='text'), ToolCallPart(tool_name='get_weather_info', args={'latitude': 26.3683, 'longitude': -80.1289}, tool_call_id='toolu_01QosMPvoXugRFQRdpscLd6p', part_kind='tool-call')], model_name='claude-3-7-sonnet-20250219', timestamp=datetime.datetime(2025, 4, 10, 10, 50, 6, 767703, tzinfo=datetime.timezone.utc), kind='response'), ModelRequest(parts=[ToolReturnPart(tool_name='get_weather_info', content={'latitude': 26.37878, 'longit

You:  And in Miami


<class 'list'>
[ModelRequest(parts=[UserPromptPart(content='And in Miami', timestamp=datetime.datetime(2025, 4, 10, 10, 53, 12, 87667, tzinfo=datetime.timezone.utc), part_kind='user-prompt')], kind='request'), ModelResponse(parts=[TextPart(content='Let me check the current temperature in Miami for you.', part_kind='text'), ToolCallPart(tool_name='get_weather_info', args={'latitude': 25.7617, 'longitude': -80.1918}, tool_call_id='toolu_01GHKV1oppJn6WQtyJScG3m7', part_kind='tool-call')], model_name='claude-3-7-sonnet-20250219', timestamp=datetime.datetime(2025, 4, 10, 10, 53, 14, 230914, tzinfo=datetime.timezone.utc), kind='response'), ModelRequest(parts=[ToolReturnPart(tool_name='get_weather_info', content={'latitude': 25.772915, 'longitude': -80.198326, 'generationtime_ms': 0.04673004150390625, 'utc_offset_seconds': 0, 'timezone': 'GMT', 'timezone_abbreviation': 'GMT', 'elevation': 5.0, 'current_units': {'time': 'iso8601', 'interval': 'seconds', 'temperature_2m': '°C', 'weather_code': 

You:  quit


In [16]:
from pydantic_ai import Agent

agent = Agent('anthropic:claude-3-7-sonnet-latest')

# First run
result1 = agent.run_sync('Who was Albert Einstein?')
print('result1:', result1.data)
#> Albert Einstein was a German-born theoretical physicist.

# Second run, passing previous messages
result2 = agent.run_sync(
    'What was his most famous equation?',
    message_history=result1.new_messages(),  
)
print('result2:', result2.data)
#> Albert Einstein's most famous equation is (E = mc^2).

result1: Albert Einstein was a German-born theoretical physicist, widely acknowledged as one of the greatest and most influential scientists of all time. He's best known for developing the theory of relativity (including the famous equation E=mc²) and for his contributions to quantum mechanics. 

Einstein was born in 1879, immigrated to the United States in 1933 to escape Nazi Germany, and died in 1955. His work fundamentally changed our understanding of physics and the universe. Beyond his scientific achievements, he was also known for his humanitarian efforts, pacifism, and advocacy for civil rights.
result2: Albert Einstein's most famous equation is E=mc², which is part of his theory of special relativity. 

This elegant equation establishes that energy (E) equals mass (m) multiplied by the speed of light (c) squared. It demonstrates the fundamental equivalence between mass and energy, showing that a small amount of mass can be converted into an enormous amount of energy. This princ

In [19]:
from dataclasses import dataclass

from pydantic_ai import Agent, RunContext


@dataclass
class User:
    name: str


agent = Agent(
    'anthropic:claude-3-7-sonnet-latest',
    deps_type=User,  
    result_type=bool,
)


@agent.system_prompt
def add_user_name(ctx: RunContext) -> str:
    print(type(ctx.deps))
    print(ctx.deps)
    return f"The user's name is {ctx.deps}."


result = agent.run_sync('Does their name start with "A"?', deps=User('Anne'))
print(result.data)

True


In [20]:
import requests
from pydantic import BaseModel
from pydantic_ai import Agent

class ResultType(BaseModel):
    temperature: float
    fun_fact_about_the_location: str

agent = Agent(
    'anthropic:claude-3-7-sonnet-latest',
    system_prompt='Be as concise as possible, reply with one sentence.',
    result_type=ResultType
)

@agent.tool_plain
def get_weather_info(latitude: float, longitude: float) -> str:
    print(f"Getting weather info for {latitude}, {longitude}")
    url = f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,weather_code"
    response = requests.get(url)
    return response.json()

message_history = []
while True:
    current_message = input('You: ')
    if current_message == 'quit':
        break
    result = agent.run_sync(current_message, message_history=message_history)
    message_history = result.new_messages()
    print(type(result.data))
    print(result.data)

You:  What's the temperature of Boca Raton, Florida


<class '__main__.ResultType'>
temperature=19.8 fun_fact_about_the_location='Boca Raton means "Rat\'s Mouth" in Spanish, but was named by mistake as early mapmakers intended to name it after the rocky inlets in the area.'


You:  quit


In [21]:
from datetime import date

from pydantic_ai import Agent, RunContext

agent = Agent(
    'anthropic:claude-3-7-sonnet-latest',
    deps_type=str,  
    system_prompt="Use the customer's name while replying to them.",  
)


@agent.system_prompt  
def add_the_users_name(ctx: RunContext[str]) -> str:
    return f"The user's name is {ctx.deps}."


@agent.system_prompt
def add_the_date() -> str:  
    return f'The date is {date.today()}.'


result = agent.run_sync('What is the date?', deps='Frank')
print(result.data)

Hello Frank! Today's date is April 10, 2025. Is there anything else I can help you with today?


In [24]:
from pydantic import BaseModel

from pydantic_ai import Agent
from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai.providers.openai import OpenAIProvider


class CityLocation(BaseModel):
    city: str
    country: str


ollama_model = OpenAIModel(
    model_name='llama3.2', provider=OpenAIProvider(base_url='http://localhost:11434/v1')
)
agent = Agent(ollama_model, result_type=CityLocation)

result = agent.run_sync('Where were the olympics held in 2012?')
print(result.data)
#> city='London' country='United Kingdom'
print(result.usage())

city='London' country='United Kingdom'
Usage(requests=1, request_tokens=176, response_tokens=24, total_tokens=200, details=None)


In [3]:
from pydantic_ai import Agent, ModelRetry

agent = Agent(
    'gemini-1.5-flash',
    system_prompt='Be very concise, reply with one sentence only.',
    retries=3
)

result = agent.run_sync('Where does "hello world" come from?')
print(result.data)

"Hello, world!" originated as a sample program in the 1978 book *The C Programming Language*.



In [4]:
result = agent.run_sync('Where does "knife" come from?')
print(result.data)

The word "knife" comes from Old English *cnif.



In [21]:
result = agent.run_sync('Where does "come" come from?')
print(result.data)

"Come" comes from the Old English word "cuman".



In [22]:
for msg in result.all_messages():
    print(msg, '\n')

ModelRequest(parts=[SystemPromptPart(content='Be very concise, reply with one sentence only.', timestamp=datetime.datetime(2025, 4, 11, 10, 37, 55, 562843, tzinfo=datetime.timezone.utc), dynamic_ref=None, part_kind='system-prompt'), UserPromptPart(content='Where does "come" come from?', timestamp=datetime.datetime(2025, 4, 11, 10, 37, 55, 562859, tzinfo=datetime.timezone.utc), part_kind='user-prompt')], kind='request') 

ModelResponse(parts=[TextPart(content='"Come" comes from the Old English word "cuman".\n', part_kind='text')], model_name='gemini-1.5-flash', timestamp=datetime.datetime(2025, 4, 11, 10, 37, 56, 177964, tzinfo=datetime.timezone.utc), kind='response') 



In [23]:
for msg in result.new_messages():
    print(msg, '\n')

ModelRequest(parts=[SystemPromptPart(content='Be very concise, reply with one sentence only.', timestamp=datetime.datetime(2025, 4, 11, 10, 37, 55, 562843, tzinfo=datetime.timezone.utc), dynamic_ref=None, part_kind='system-prompt'), UserPromptPart(content='Where does "come" come from?', timestamp=datetime.datetime(2025, 4, 11, 10, 37, 55, 562859, tzinfo=datetime.timezone.utc), part_kind='user-prompt')], kind='request') 

ModelResponse(parts=[TextPart(content='"Come" comes from the Old English word "cuman".\n', part_kind='text')], model_name='gemini-1.5-flash', timestamp=datetime.datetime(2025, 4, 11, 10, 37, 56, 177964, tzinfo=datetime.timezone.utc), kind='response') 



In [24]:
result1 = agent.run_sync('Explain it in details', message_history=result.new_messages())
print(result1.data)

The English word "come" originates from the Old English verb "cuman," which shares ancestry with other Germanic languages' words for "to come," reflecting a common Proto-Germanic root *kwem-.  This Proto-Germanic root further traces back to a Proto-Indo-European root, *gʷem-, indicating a deep and ancient linguistic connection across numerous language families.



In [26]:
for msg in result1.all_messages():
    print(msg, '\n')

ModelRequest(parts=[SystemPromptPart(content='Be very concise, reply with one sentence only.', timestamp=datetime.datetime(2025, 4, 11, 10, 37, 55, 562843, tzinfo=datetime.timezone.utc), dynamic_ref=None, part_kind='system-prompt'), UserPromptPart(content='Where does "come" come from?', timestamp=datetime.datetime(2025, 4, 11, 10, 37, 55, 562859, tzinfo=datetime.timezone.utc), part_kind='user-prompt')], kind='request') 

ModelResponse(parts=[TextPart(content='"Come" comes from the Old English word "cuman".\n', part_kind='text')], model_name='gemini-1.5-flash', timestamp=datetime.datetime(2025, 4, 11, 10, 37, 56, 177964, tzinfo=datetime.timezone.utc), kind='response') 

ModelRequest(parts=[UserPromptPart(content='Explain it in details', timestamp=datetime.datetime(2025, 4, 11, 10, 38, 26, 16970, tzinfo=datetime.timezone.utc), part_kind='user-prompt')], kind='request') 

ModelResponse(parts=[TextPart(content='The English word "come" originates from the Old English verb "cuman," which sha

In [27]:
for msg in result1.new_messages():
    print(msg, '\n')

ModelRequest(parts=[UserPromptPart(content='Explain it in details', timestamp=datetime.datetime(2025, 4, 11, 10, 38, 26, 16970, tzinfo=datetime.timezone.utc), part_kind='user-prompt')], kind='request') 

ModelResponse(parts=[TextPart(content='The English word "come" originates from the Old English verb "cuman," which shares ancestry with other Germanic languages\' words for "to come," reflecting a common Proto-Germanic root *kwem-.  This Proto-Germanic root further traces back to a Proto-Indo-European root, *gʷem-, indicating a deep and ancient linguistic connection across numerous language families.\n', part_kind='text')], model_name='gemini-1.5-flash', timestamp=datetime.datetime(2025, 4, 11, 10, 38, 27, 56639, tzinfo=datetime.timezone.utc), kind='response') 



In [29]:
result1.all_messages_json()

b'[{"parts":[{"content":"Be very concise, reply with one sentence only.","timestamp":"2025-04-11T10:37:55.562843Z","dynamic_ref":null,"part_kind":"system-prompt"},{"content":"Where does \\"come\\" come from?","timestamp":"2025-04-11T10:37:55.562859Z","part_kind":"user-prompt"}],"kind":"request"},{"parts":[{"content":"\\"Come\\" comes from the Old English word \\"cuman\\".\\n","part_kind":"text"}],"model_name":"gemini-1.5-flash","timestamp":"2025-04-11T10:37:56.177964Z","kind":"response"},{"parts":[{"content":"Explain it in details","timestamp":"2025-04-11T10:38:26.016970Z","part_kind":"user-prompt"}],"kind":"request"},{"parts":[{"content":"The English word \\"come\\" originates from the Old English verb \\"cuman,\\" which shares ancestry with other Germanic languages\' words for \\"to come,\\" reflecting a common Proto-Germanic root *kwem-.  This Proto-Germanic root further traces back to a Proto-Indo-European root, *g\xca\xb7em-, indicating a deep and ancient linguistic connection acr

In [4]:
from pydantic_ai import Agent
from dotenv import load_dotenv

agent = Agent('anthropic:claude-3-7-sonnet-latest', system_prompt='Be as concise as possible, reply with one sentence.')

message_history = []
while True:
    current_message = input('You: ')
    if current_message == 'quit':
        break
    result = agent.run_sync(current_message, message_history=message_history)
    message_history = result.new_messages()
    print(type(message_history))
    print(message_history)
    print(result.data)

You:  My name is Juan. I'd like to know what's the capital of France


<class 'list'>
[ModelRequest(parts=[SystemPromptPart(content='Be as concise as possible, reply with one sentence.', timestamp=datetime.datetime(2025, 5, 16, 20, 32, 56, 847400, tzinfo=datetime.timezone.utc), dynamic_ref=None, part_kind='system-prompt'), UserPromptPart(content="My name is Juan. I'd like to know what's the capital of France", timestamp=datetime.datetime(2025, 5, 16, 20, 32, 56, 847476, tzinfo=datetime.timezone.utc), part_kind='user-prompt')], kind='request'), ModelResponse(parts=[TextPart(content='The capital of France is Paris.', part_kind='text')], model_name='claude-3-7-sonnet-latest', timestamp=datetime.datetime(2025, 5, 16, 20, 32, 58, 833665, tzinfo=datetime.timezone.utc), kind='response')]
The capital of France is Paris.


You:  Tell me a story about that city


<class 'list'>
[ModelRequest(parts=[UserPromptPart(content='Tell me a story about that city', timestamp=datetime.datetime(2025, 5, 16, 20, 33, 35, 41242, tzinfo=datetime.timezone.utc), part_kind='user-prompt')], kind='request'), ModelResponse(parts=[TextPart(content='Paris, the City of Light, grew from a small island settlement on the Seine to become a global cultural icon, captivating visitors with its magnificent architecture, world-renowned cuisine, and artistic heritage that continues to inspire dreamers from around the world.', part_kind='text')], model_name='claude-3-7-sonnet-latest', timestamp=datetime.datetime(2025, 5, 16, 20, 33, 37, 270231, tzinfo=datetime.timezone.utc), kind='response')]
Paris, the City of Light, grew from a small island settlement on the Seine to become a global cultural icon, captivating visitors with its magnificent architecture, world-renowned cuisine, and artistic heritage that continues to inspire dreamers from around the world.


You:  What's my name


<class 'list'>
[ModelRequest(parts=[UserPromptPart(content="What's my name", timestamp=datetime.datetime(2025, 5, 16, 20, 33, 56, 961658, tzinfo=datetime.timezone.utc), part_kind='user-prompt')], kind='request'), ModelResponse(parts=[TextPart(content="I don't know your name as you haven't shared it with me. If you'd like me to address you by your name in our conversation, feel free to introduce yourself, and I'll be happy to use it.", part_kind='text')], model_name='claude-3-7-sonnet-latest', timestamp=datetime.datetime(2025, 5, 16, 20, 33, 58, 855438, tzinfo=datetime.timezone.utc), kind='response')]
I don't know your name as you haven't shared it with me. If you'd like me to address you by your name in our conversation, feel free to introduce yourself, and I'll be happy to use it.


You:  quit


In [5]:
from pydantic_ai import Agent
from dotenv import load_dotenv

agent = Agent('anthropic:claude-3-7-sonnet-latest', system_prompt='Be as concise as possible, reply with one sentence.')

message_history = []
while True:
    current_message = input('You: ')
    if current_message == 'quit':
        break
    result = agent.run_sync(current_message, message_history=message_history)
    message_history = result.all_messages()
    print(type(message_history))
    print(message_history)
    print(result.data)

You:  My name is Juan, I'd like to know more about France


<class 'list'>
[ModelRequest(parts=[SystemPromptPart(content='Be as concise as possible, reply with one sentence.', timestamp=datetime.datetime(2025, 5, 16, 20, 39, 47, 112629, tzinfo=datetime.timezone.utc), dynamic_ref=None, part_kind='system-prompt'), UserPromptPart(content="My name is Juan, I'd like to know more about France", timestamp=datetime.datetime(2025, 5, 16, 20, 39, 47, 112693, tzinfo=datetime.timezone.utc), part_kind='user-prompt')], kind='request'), ModelResponse(parts=[TextPart(content='France is a Western European country known for its art, culture, cuisine, and landmarks like the Eiffel Tower, Louvre Museum, and Palace of Versailles.', part_kind='text')], model_name='claude-3-7-sonnet-latest', timestamp=datetime.datetime(2025, 5, 16, 20, 39, 48, 841645, tzinfo=datetime.timezone.utc), kind='response')]
France is a Western European country known for its art, culture, cuisine, and landmarks like the Eiffel Tower, Louvre Museum, and Palace of Versailles.


You:  What's the capital


<class 'list'>
[ModelRequest(parts=[SystemPromptPart(content='Be as concise as possible, reply with one sentence.', timestamp=datetime.datetime(2025, 5, 16, 20, 39, 47, 112629, tzinfo=datetime.timezone.utc), dynamic_ref=None, part_kind='system-prompt'), UserPromptPart(content="My name is Juan, I'd like to know more about France", timestamp=datetime.datetime(2025, 5, 16, 20, 39, 47, 112693, tzinfo=datetime.timezone.utc), part_kind='user-prompt')], kind='request'), ModelResponse(parts=[TextPart(content='France is a Western European country known for its art, culture, cuisine, and landmarks like the Eiffel Tower, Louvre Museum, and Palace of Versailles.', part_kind='text')], model_name='claude-3-7-sonnet-latest', timestamp=datetime.datetime(2025, 5, 16, 20, 39, 48, 841645, tzinfo=datetime.timezone.utc), kind='response'), ModelRequest(parts=[UserPromptPart(content="What's the capital", timestamp=datetime.datetime(2025, 5, 16, 20, 40, 13, 519935, tzinfo=datetime.timezone.utc), part_kind='us

You:  What's my name


<class 'list'>
[ModelRequest(parts=[SystemPromptPart(content='Be as concise as possible, reply with one sentence.', timestamp=datetime.datetime(2025, 5, 16, 20, 39, 47, 112629, tzinfo=datetime.timezone.utc), dynamic_ref=None, part_kind='system-prompt'), UserPromptPart(content="My name is Juan, I'd like to know more about France", timestamp=datetime.datetime(2025, 5, 16, 20, 39, 47, 112693, tzinfo=datetime.timezone.utc), part_kind='user-prompt')], kind='request'), ModelResponse(parts=[TextPart(content='France is a Western European country known for its art, culture, cuisine, and landmarks like the Eiffel Tower, Louvre Museum, and Palace of Versailles.', part_kind='text')], model_name='claude-3-7-sonnet-latest', timestamp=datetime.datetime(2025, 5, 16, 20, 39, 48, 841645, tzinfo=datetime.timezone.utc), kind='response'), ModelRequest(parts=[UserPromptPart(content="What's the capital", timestamp=datetime.datetime(2025, 5, 16, 20, 40, 13, 519935, tzinfo=datetime.timezone.utc), part_kind='us

You:  quit


In [7]:
from pydantic_ai import Agent
from dotenv import load_dotenv
from datetime import datetime

agent = Agent('anthropic:claude-3-7-sonnet-latest', system_prompt='Be as concise as possible, reply with one sentence.')

@agent.tool_plain
def get_current_time():
    """Returns the current time in 12-hour format with AM/PM indicator.
    
    Returns:
        str: The current time formatted as HH:MM AM/PM
    """
    return datetime.now().strftime("%I:%M %p")

message_history = []
while True:
    current_message = input('You: ')
    if current_message == 'quit':
        break
    result = agent.run_sync(current_message, message_history=message_history)
    message_history = result.all_messages()
    print(type(message_history))
    print(message_history)
    print(result.data)

You:  What's the current time


<class 'list'>
[ModelRequest(parts=[SystemPromptPart(content='Be as concise as possible, reply with one sentence.', timestamp=datetime.datetime(2025, 5, 16, 20, 45, 10, 892800, tzinfo=datetime.timezone.utc), dynamic_ref=None, part_kind='system-prompt'), UserPromptPart(content="What's the current time", timestamp=datetime.datetime(2025, 5, 16, 20, 45, 10, 892815, tzinfo=datetime.timezone.utc), part_kind='user-prompt')], kind='request'), ModelResponse(parts=[ToolCallPart(tool_name='get_current_time', args={}, tool_call_id='toolu_017G19RqEoYfNwfXzY1Pb26Q', part_kind='tool-call')], model_name='claude-3-7-sonnet-latest', timestamp=datetime.datetime(2025, 5, 16, 20, 45, 14, 47044, tzinfo=datetime.timezone.utc), kind='response'), ModelRequest(parts=[ToolReturnPart(tool_name='get_current_time', content='04:45 PM', tool_call_id='toolu_017G19RqEoYfNwfXzY1Pb26Q', timestamp=datetime.datetime(2025, 5, 16, 20, 45, 14, 47789, tzinfo=datetime.timezone.utc), part_kind='tool-return')], kind='request'), 

You:  quit
